# Food Delivery Data Analysis

## Objective
The objective of this project is to analyze food delivery data by combining
transactional order data, user information, and restaurant details.

The analysis focuses on:
- User behavior
- City-wise revenue
- Membership impact (Gold vs Regular)
- Restaurant performance


In [1]:
import pandas as pd
import sqlite3


In [2]:
orders = pd.read_csv("orders.csv")
orders.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [3]:
# Load users data from JSON
users = pd.read_json("users.json")

# Display first few rows
users.head()


,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [4]:
# Create an in-memory SQLite database
conn = sqlite3.connect(":memory:")

# Read and execute the SQL file
with open("restaurants.sql", "r") as file:
    sql_script = file.read()

conn.executescript(sql_script)

# Load restaurants table into a DataFrame
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

# Display first few rows
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [5]:
# Merge orders with users data using LEFT JOIN
orders_users_df = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)

# Preview merged data
orders_users_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [6]:
# Merge orders+users data with restaurants data using LEFT JOIN
final_df = pd.merge(
    orders_users_df,
    restaurants,
    on="restaurant_id",
    how="left"
)

# Preview final merged dataset
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [7]:
# Save the final merged dataset to CSV
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [8]:
# Total number of orders placed by Gold members
gold_orders_count = final_df[final_df["membership"] == "Gold"].shape[0]
gold_orders_count


4987

In [9]:
# Total revenue generated from orders placed in Hyderabad city
hyderabad_revenue = round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)
hyderabad_revenue


1889367

In [10]:
# Number of distinct users who placed at least one order
distinct_users_count = final_df["user_id"].nunique()
distinct_users_count


2883

In [11]:
# Average order value for Gold members (rounded to 2 decimals)
avg_gold_order_value = round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(), 2
)
avg_gold_order_value


np.float64(797.15)

In [12]:
# Number of orders placed for restaurants with rating >= 4.5
high_rating_orders = final_df[final_df["rating"] >= 4.5].shape[0]
high_rating_orders


3374

In [13]:
# Identify the top revenue city among Gold members
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

# Count number of orders in that city for Gold members
top_city_gold_orders = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
].shape[0]

top_city_gold_orders


1337

## Conclusion
The datasets were successfully merged using LEFT JOINs.
The analysis provided insights into user behavior, membership impact,
city-wise revenue, and restaurant performance.
